# Pytorch Geometric GNN - homogen

## Import section

In [1]:
from data_gen import data_transform_split

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

## Section for GNN with rdkit

In [ ]:
data_rdkit = data_transform_split(2)

Load CSV
Split part
Begin data creation part
Build trainset


0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [2]:
import pandas as pd
df = pd.read_csv("df_assay_entries.csv")

In [5]:
from rdkit.Chem import Descriptors, MolFromSmiles

In [3]:
zwi = df[['cid', 'smiles']].drop_duplicates(subset=['cid'])

In [6]:
zwi['molecule']=zwi['smiles'].map(lambda x: MolFromSmiles(x))

In [9]:
zwi2 = zwi.sort_values(by=['cid'])['molecule'].map(lambda x: [func(x) for _, func in Descriptors.descList])

KeyboardInterrupt: 